In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
print(tf.__version__)

2.4.1


In [ ]:
train_ds = tfds.load('colorectal_histology', split='train[:80%]', shuffle_files=True, as_supervised=True)
train_ds = train_ds.batch(32)
train_ds = train_ds.map(lambda img, label: (tf.image.convert_image_dtype(img,dtype=tf.float32), label))

In [ ]:
val_ds = tfds.load('colorectal_histology', split='train[80%:90%]', shuffle_files=True, as_supervised=True)
val_ds = val_ds.batch(32)
val_ds = val_ds.map(lambda img, label: (tf.image.convert_image_dtype(img,dtype=tf.float32), label))

In [ ]:
test_ds = tfds.load('colorectal_histology', split='train[90%:100%]', shuffle_files=True, as_supervised=True)
test_ds = test_ds.batch(32)
test_ds = test_ds.map(lambda img, label: (tf.image.convert_image_dtype(img,dtype=tf.float32), label))

In [2]:
train_ds, val_ds, test_ds = tfds.load('colorectal_histology', split=['train[:80%]','train[80%:90%]','train[90%:100%]'], shuffle_files=True, as_supervised=True)
train_ds = train_ds.batch(32)
train_ds = train_ds.map(lambda img, label: (tf.image.convert_image_dtype(img,dtype=tf.float32), label))
val_ds = val_ds.batch(32)
val_ds = val_ds.map(lambda img, label: (tf.image.convert_image_dtype(img,dtype=tf.float32), label))
test_ds = test_ds.batch(32)
test_ds = test_ds.map(lambda img, label: (tf.image.convert_image_dtype(img,dtype=tf.float32), label))

In [3]:
customModel = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(150, 150, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
customModel.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
customModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 67500)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8640128   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 8,641,418
Trainable params: 8,641,418
Non-trainable params: 0
_________________________________________________________________


In [4]:
customModelHistory = customModel.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
)

Epoch 1/10
125/125 [==============================] - 5s 40ms/step - loss: 29.4184 - sparse_categorical_accuracy: 0.1885 - val_loss: 4.4191 - val_sparse_categorical_accuracy: 0.2640
Epoch 2/10
125/125 [==============================] - 5s 36ms/step - loss: 4.1430 - sparse_categorical_accuracy: 0.3358 - val_loss: 3.7383 - val_sparse_categorical_accuracy: 0.3380
Epoch 3/10
125/125 [==============================] - 5s 37ms/step - loss: 3.1154 - sparse_categorical_accuracy: 0.3801 - val_loss: 3.2498 - val_sparse_categorical_accuracy: 0.2720
Epoch 4/10
125/125 [==============================] - 5s 42ms/step - loss: 2.3904 - sparse_categorical_accuracy: 0.4185 - val_loss: 1.7761 - val_sparse_categorical_accuracy: 0.4040
Epoch 5/10
125/125 [==============================] - 5s 40ms/step - loss: 2.2665 - sparse_categorical_accuracy: 0.4570 - val_loss: 2.9370 - val_sparse_categorical_accuracy: 0.3560
Epoch 6/10
125/125 [==============================] - 5s 40ms/step - loss: 2.2820 - sparse_cat

In [7]:
print(customModelHistory.history['val_sparse_categorical_accuracy'])

[0.2639999985694885, 0.33799999952316284, 0.2720000147819519, 0.40400001406669617, 0.35600000619888306, 0.44200000166893005, 0.3959999978542328, 0.5120000243186951, 0.4740000069141388, 0.4959999918937683]


In [8]:
vggModel = tf.keras.applications.VGG16(
    weights=None, input_shape=(150,150,3)
)
vggModel.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
vggModel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [10]:
vggModelHistory = vggModel.fit(
    train_ds,
    epochs=1,
    validation_data=val_ds,
)

125/125 [==============================] - 1378s 11s/step - loss: 3.3491 - sparse_categorical_accuracy: 0.1141 - val_loss: 2.1281 - val_sparse_categorical_accuracy: 0.0900
